In [3]:
from source_code.ConfigManager import ConfigManager
from source_code.LLM_Module import LLM_Module
from source_code.PreferencesModule import PreferencesModule
from source_code.RecommenderModule import RecommenderModule
from source_code.SentimentAnalysis import SentimentAnalysisModule
from source_code.Tokenizer import TokenizerModule
from source_code.Frontend import FrontendImplementation
from source_code.IntentRecognition import IntentRecognitionModule
from source_code.UserDataManagementModule import UserDataManagementModule


class ApplicationPipeline:

    def __init__(self, frontend: FrontendImplementation = None):
        self.intentModule = IntentRecognitionModule()
        self.tokenizer = TokenizerModule()
        self.userDataManagementModule = UserDataManagementModule()
        self.sentimentModule = SentimentAnalysisModule()
        self.preferencesModule = PreferencesModule()
        self.LLM_Module = LLM_Module()

        # Initialize RecommenderModule with configuration
        product_file = ConfigManager.CONFIG.get('product_file', 'PRODUCT_clean.csv')
        api_url = ConfigManager.CONFIG.get('recommendation_api_url', 'http://localhost:8000')
        self.recommenderModule = RecommenderModule(api_url=api_url, product_file=product_file)

        self.frontend = frontend
        self.frontend.events.on_data += self.run
        self.frontend.run()

    def run(self, msg):
        # Intent recognition block
        intent = self.intentModule.recognize_intent(msg)

        if ConfigManager.CONFIG.get('debug', False):
            self.frontend.insert_message(intent, 'System - Intent Module')

        # Message tokenization block
        tokens = ' '.join(self.tokenizer.tokenize(msg, intent))

        # Intent-based computation flow
        if intent == 'Unrelated':
            self.frontend.insert_message(self.LLM_Module.response(intent, msg), 'Chatbot')
            return 'Unrelated'
        
        if intent == 'Uncovered':
            self.frontend.insert_message(self.LLM_Module.response(intent, msg), 'Chatbot')
            return 'Uncovered'
        
        if intent == 'Purchase' or intent == 'Self-identifying':
            response = self.userDataManagementModule.process_intent(intent, tokens)

        if intent == 'Preference':
            # Sentiment recognition block
            sentiment = self.sentimentModule.analyze(msg)
            if ConfigManager.CONFIG.get('debug', False):
                self.frontend.insert_message(sentiment, 'System - Sentiment Module')

            # Preferences update block
            user_id = self.userDataManagementModule.get_user_id()
            response = self.preferencesModule.update_preferences(user_id, tokens, sentiment)
            if ConfigManager.CONFIG.get('debug', False):
                self.frontend.insert_message( 'Product preferences have been updated', 'System - Preference Module')

        if intent == 'Recommendation':
            
            user_id = self.userDataManagementModule.get_user_id()
            purchases_file = f"data/user_data/{user_id}purchases.csv"

            response = self.recommenderModule.get_recommendations(purchases_file=purchases_file, user_id=user_id, k=5)

        self.frontend.insert_message(self.LLM_Module.response(intent, f'The user stated:' + msg + " Computation status: " + response['status'] + ", " + response['message']), 'Chatbot')


In [4]:
from source_code.ConfigManager import load_config
from source_code.Frontend import FrontendImplementation

# Loading the configurations from file
load_config()

if __name__ == "__main__":
    app = ApplicationPipeline(FrontendImplementation())
    

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
C:\Users\nvisc\miniconda3\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 1.0}


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
C:\Users\nvisc\miniconda3\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


...

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.82}


[nltk_data] Downloading package names to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nvisc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is al

TclError: invalid command name ".!label2.!entry"